In [1]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from PIL import Image, ImageTk, ImageOps
import cv2
import numpy as np
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

class AdvancedImageProcessor:
    def __init__(self, root):
        self.root = root
        
        # فرهنگ لغت برای متون واسط کاربری در دو زبان: فارسی (fa) و انگلیسی (en)
        self.i18n = {
            "fa": {
                "app_title": "پردازش تصویر پیشرفته",
                "load_images": "بارگذاری تصاویر",
                "reset_settings": "ریست تنظیمات",
                "save_processed_image": "ذخیره تصویر پردازش‌شده",
                
                "advanced_settings": "تنظیمات پیشرفته",
                "brightness": "روشنایی:",
                "contrast": "کنتراست:",
                "filters": "فیلتر‌ها:",
                "kernel_size": "اندازه کرنل:",
                "threshold_title": "آستانه:",
                "rotation_title": "چرخش:",
                "rotation_90": "چرخش 90°",
                "rotation_180": "چرخش 180°",
                "rotation_270": "چرخش 270°",
                "flip_title": "پهلوی تصویر:",
                "horizontal_flip": "پهلوی افقی",
                "vertical_flip": "پهلوی عمودی",
                
                "original_image": "تصویر اصلی",
                "processed_image": "تصویر پردازش شده",
                "histogram": "هیستوگرام",
                "filtered_image": "تصویر فیلتر شده",
                
                "apply_changes": "اعمال تغییرات",
                
                "warning": "هشدار",
                "error": "خطا",
                "success": "موفقیت",
                
                "image_already_loaded": "تصویر {title} قبلاً بارگذاری شده است.",
                "image_load_failed": "بارگذاری تصویر از مسیر {path} موفقیت‌آمیز نبود.",
                "no_image_to_save": "هیچ تصویری برای ذخیره وجود ندارد.",
                "image_saved_successfully": "تصویر با موفقیت در مسیر {file_path} ذخیره شد.",
                
                "hue": "Hue",
                "saturation": "Saturation",
                "value": "Value",
                
                "red": "قرمز",
                "green": "سبز",
                "blue": "آبی",
                
                "apply_model_changes": "اعمال تغییرات",
            },
            "en": {
                "app_title": "Advanced Image Processing",
                "load_images": "Load Images",
                "reset_settings": "Reset Settings",
                "save_processed_image": "Save Processed Image",
                
                "advanced_settings": "Advanced Settings",
                "brightness": "Brightness:",
                "contrast": "Contrast:",
                "filters": "Filters:",
                "kernel_size": "Kernel Size:",
                "threshold_title": "Threshold:",
                "rotation_title": "Rotate:",
                "rotation_90": "Rotate 90°",
                "rotation_180": "Rotate 180°",
                "rotation_270": "Rotate 270°",
                "flip_title": "Flip Image:",
                "horizontal_flip": "Horizontal Flip",
                "vertical_flip": "Vertical Flip",
                
                "original_image": "Original Image",
                "processed_image": "Processed Image",
                "histogram": "Histogram",
                "filtered_image": "Filtered Image",
                
                "apply_changes": "Apply Changes",
                
                "warning": "Warning",
                "error": "Error",
                "success": "Success",
                
                "image_already_loaded": "Image {title} is already loaded.",
                "image_load_failed": "Could not load image from path {path}.",
                "no_image_to_save": "No image available to save.",
                "image_saved_successfully": "Image was saved successfully to {file_path}.",
                
                "hue": "Hue",
                "saturation": "Saturation",
                "value": "Value",
                
                "red": "Red",
                "green": "Green",
                "blue": "Blue",
                
                "apply_model_changes": "Apply Changes",
            }
        }
        
        # زبان فعلی
        self.current_language = "fa"
        
        # عنوان پنجره بر اساس زبان پیش‌فرض
        self.root.title(self.i18n[self.current_language]["app_title"])
        self.root.geometry("1600x900")
        
        self.images = {}
        self.current_tab = None
        
        # مدل‌های رنگی (در این مثال نام‌ها را ثابت گذاشته‌ایم)
        self.color_models = [
            "RGB", "HSV", "LAB", "YCbCr",
            "HSL", "XYZ", "Grayscale", "CMYK"
        ]
        
        # ایجاد اجزای رابط کاربری
        self.create_widgets()
        
    def create_widgets(self):
        # نوت‌بوک برای تب‌های تصویر
        self.notebook = ttk.Notebook(self.root)
        self.notebook.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        self.notebook.bind("<<NotebookTabChanged>>", self.tab_changed)
        
        # فریم کنترل‌ها
        control_frame = ttk.Frame(self.root)
        control_frame.pack(fill=tk.X, padx=10, pady=5)
        
        # دکمه‌های کنترل (ابتدا ایجاد می‌کنیم ولی متنشان را بعداً در تابع update_ui_texts قرار می‌دهیم)
        self.btn_load_images = ttk.Button(control_frame, command=self.load_images)
        self.btn_load_images.pack(side=tk.LEFT, padx=5)
        
        self.btn_reset_settings = ttk.Button(control_frame, command=self.reset_settings)
        self.btn_reset_settings.pack(side=tk.LEFT, padx=5)
        
        self.btn_save_image = ttk.Button(control_frame, command=self.save_image)
        self.btn_save_image.pack(side=tk.LEFT, padx=5)
        
        # انتخاب زبان (دو دکمه: فارسی و انگلیسی)
        self.btn_lang_fa = ttk.Button(control_frame, text="FA", command=lambda: self.set_language("fa"))
        self.btn_lang_fa.pack(side=tk.LEFT, padx=5)
        
        self.btn_lang_en = ttk.Button(control_frame, text="EN", command=lambda: self.set_language("en"))
        self.btn_lang_en.pack(side=tk.LEFT, padx=5)
        
        # انتخاب مدل رنگی
        self.color_model_var = tk.StringVar()
        self.model_dropdown = ttk.Combobox(
            control_frame, 
            textvariable=self.color_model_var,
            values=self.color_models,
            state="readonly"
        )
        self.model_dropdown.pack(side=tk.LEFT, padx=5)
        self.model_dropdown.bind("<<ComboboxSelected>>", self.update_controls)
        
        # فریم تنظیمات (برای اسلایدرهای مدل رنگی)
        self.settings_frame = ttk.Frame(self.root)
        self.settings_frame.pack(fill=tk.X, padx=10, pady=5)
        
        # فریم تنظیمات پیشرفته
        self.advanced_settings_frame = ttk.LabelFrame(self.root)
        self.advanced_settings_frame.pack(fill=tk.X, padx=10, pady=5)
        
        # در تابع update_ui_texts عنوان این LabelFrame را عوض می‌کنیم
        # ایجاد کنترل‌های پیشرفته (روشنایی، کنتراست و...)
        self.create_advanced_controls()
        
        # فریم نمایش تصاویر
        self.display_frame = ttk.Frame(self.root)
        self.display_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)
        
        # ایجاد بخش‌های نمایش
        self.create_display_areas()
        
        # در انتها متن تمام ویجت‌ها را بر اساس زبان فعلی به‌روزرسانی می‌کنیم
        self.update_ui_texts()
        
    def set_language(self, lang):
        """تنظیم زبان و به‌روزرسانی رابط کاربری"""
        self.current_language = lang
        self.update_ui_texts()
        
    def update_ui_texts(self):
        """بر اساس self.current_language، متن تمام ویجت‌ها را به‌روزرسانی می‌کند."""
        # تغییر عنوان برنامه
        self.root.title(self.i18n[self.current_language]["app_title"])
        
        # دکمه‌های کنترل
        self.btn_load_images.config(text=self.i18n[self.current_language]["load_images"])
        self.btn_reset_settings.config(text=self.i18n[self.current_language]["reset_settings"])
        self.btn_save_image.config(text=self.i18n[self.current_language]["save_processed_image"])
        
        # لیبل‌فریم تنظیمات پیشرفته
        self.advanced_settings_frame.config(text=self.i18n[self.current_language]["advanced_settings"])
        
        # برچسب‌های اسلایدر روشنایی و کنتراست
        self.brightness_label.config(text=self.i18n[self.current_language]["brightness"])
        self.contrast_label.config(text=self.i18n[self.current_language]["contrast"])
        
        # برچسب فیلتر
        self.filter_label.config(text=self.i18n[self.current_language]["filters"])
        
        # برچسب Threshold frame
        self.threshold_label.config(text=self.i18n[self.current_language]["threshold_title"])
        
        # برچسب چرخش
        self.rotate_label.config(text=self.i18n[self.current_language]["rotation_title"])
        self.btn_rotate_90.config(text=self.i18n[self.current_language]["rotation_90"])
        self.btn_rotate_180.config(text=self.i18n[self.current_language]["rotation_180"])
        self.btn_rotate_270.config(text=self.i18n[self.current_language]["rotation_270"])
        
        # برچسب پهلو چرخاندن
        self.flip_label.config(text=self.i18n[self.current_language]["flip_title"])
        self.btn_flip_horizontal.config(text=self.i18n[self.current_language]["horizontal_flip"])
        self.btn_flip_vertical.config(text=self.i18n[self.current_language]["vertical_flip"])
        
        # فریم‌های نمایش
        self.original_frame.config(text=self.i18n[self.current_language]["original_image"])
        self.processed_frame.config(text=self.i18n[self.current_language]["processed_image"])
        self.hist_frame.config(text=self.i18n[self.current_language]["histogram"])
        self.advanced_frame.config(text=self.i18n[self.current_language]["filtered_image"])
        
        # اگر بخواهید اسلایدرهای هر مدل رنگی هم حین تغییر زبان آپدیت شوند،
        # می‌توانید از همین روش (نگه داشتن مراجع به ویجت‌ها) استفاده کنید.
        # در این مثال برای سادگی، فقط برچسب‌های ثابت را تغییر داده‌ایم.
        
        # دکمه اعمال تغییرات (در انتهای update_controls ساخته می‌شود)
        # اگر می‌خواهید همیشه وجود داشته باشد، مرجع آن را ذخیره کرده و اینجا تغییر دهید.
        if hasattr(self, 'btn_apply_changes'):
            self.btn_apply_changes.config(text=self.i18n[self.current_language]["apply_model_changes"])
    
    def create_display_areas(self):
        # تنظیمات گرید
        self.display_frame.columnconfigure(0, weight=1)
        self.display_frame.columnconfigure(1, weight=1)
        self.display_frame.columnconfigure(2, weight=1)
        self.display_frame.columnconfigure(3, weight=1)
        self.display_frame.rowconfigure(0, weight=1)
        
        # تصویر اصلی
        self.original_frame = ttk.LabelFrame(self.display_frame)
        self.original_frame.grid(row=0, column=0, padx=5, pady=5, sticky="nsew")
        self.original_label = ttk.Label(self.original_frame)
        self.original_label.pack(expand=True)
        
        # تصویر پردازش شده
        self.processed_frame = ttk.LabelFrame(self.display_frame)
        self.processed_frame.grid(row=0, column=1, padx=5, pady=5, sticky="nsew")
        self.processed_label = ttk.Label(self.processed_frame)
        self.processed_label.pack(expand=True)
        
        # هیستوگرام
        self.hist_frame = ttk.LabelFrame(self.display_frame)
        self.hist_frame.grid(row=0, column=2, padx=5, pady=5, sticky="nsew")
        self.hist_canvas = None
        
        # تصویر پیشرفته (مانند تصویر فیلتر شده)
        self.advanced_frame = ttk.LabelFrame(self.display_frame)
        self.advanced_frame.grid(row=0, column=3, padx=5, pady=5, sticky="nsew")
        self.advanced_label = ttk.Label(self.advanced_frame)
        self.advanced_label.pack(expand=True)
    
    def create_advanced_controls(self):
        # تنظیمات روشنایی
        self.brightness_label = ttk.Label(self.advanced_settings_frame)
        self.brightness_label.grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.brightness_var = tk.IntVar(value=0)
        brightness_slider = ttk.Scale(
            self.advanced_settings_frame,
            from_=-100, to=100,
            variable=self.brightness_var,
            command=lambda e: self.process_image()
        )
        brightness_slider.grid(row=0, column=1, padx=5, pady=5, sticky="ew")
        self.advanced_settings_frame.columnconfigure(1, weight=1)
        
        # تنظیمات کنتراست
        self.contrast_label = ttk.Label(self.advanced_settings_frame)
        self.contrast_label.grid(row=1, column=0, padx=5, pady=5, sticky="w")
        self.contrast_var = tk.DoubleVar(value=1.0)
        contrast_slider = ttk.Scale(
            self.advanced_settings_frame,
            from_=0.5, to=3.0,
            variable=self.contrast_var,
            command=lambda e: self.process_image()
        )
        contrast_slider.grid(row=1, column=1, padx=5, pady=5, sticky="ew")
        
        # فیلترها
        self.filter_label = ttk.Label(self.advanced_settings_frame)
        self.filter_label.grid(row=2, column=0, padx=5, pady=5, sticky="w")
        
        self.filter_var = tk.StringVar()
        self.filter_dropdown = ttk.Combobox(
            self.advanced_settings_frame,
            textvariable=self.filter_var,
            values=["None", "Gaussian Blur", "Median Blur", "Canny Edge", "Sharpen", "Sepia", 
                    "Emboss", "Histogram Equalization", "Thresholding"],
            state="readonly"
        )
        self.filter_dropdown.current(0)
        self.filter_dropdown.grid(row=2, column=1, padx=5, pady=5, sticky="ew")
        self.filter_dropdown.bind("<<ComboboxSelected>>", self.filter_changed)
        
        # فریم‌های مخصوص فیلترها
        self.create_filter_settings()
        
        # فریم آستانه
        self.threshold_frame = ttk.Frame(self.advanced_settings_frame)
        self.threshold_label = ttk.Label(self.threshold_frame)  # برچسب آستانه
        self.threshold_label.pack(side=tk.LEFT, padx=5)
        self.threshold_var = tk.IntVar(value=128)
        self.threshold_slider = ttk.Scale(
            self.threshold_frame,
            from_=0, to=255,
            variable=self.threshold_var,
            command=lambda e: self.process_image()
        )
        self.threshold_slider.set(128)
        self.threshold_slider.pack(side=tk.LEFT, fill=tk.X, expand=True)
        # ابتدا پنهان است
        self.threshold_frame.grid_remove()
        
        # بخش چرخش
        self.rotate_label = ttk.Label(self.advanced_settings_frame)
        self.rotate_label.grid(row=4, column=0, padx=5, pady=5, sticky="w")
        
        rotate_frame = ttk.Frame(self.advanced_settings_frame)
        rotate_frame.grid(row=4, column=1, padx=5, pady=5, sticky="w")
        self.btn_rotate_90 = ttk.Button(rotate_frame, command=lambda: self.rotate_image(90))
        self.btn_rotate_90.pack(side=tk.LEFT, padx=2)
        self.btn_rotate_180 = ttk.Button(rotate_frame, command=lambda: self.rotate_image(180))
        self.btn_rotate_180.pack(side=tk.LEFT, padx=2)
        self.btn_rotate_270 = ttk.Button(rotate_frame, command=lambda: self.rotate_image(270))
        self.btn_rotate_270.pack(side=tk.LEFT, padx=2)
        
        # بخش پهلو چرخاندن
        self.flip_label = ttk.Label(self.advanced_settings_frame)
        self.flip_label.grid(row=5, column=0, padx=5, pady=5, sticky="w")
        
        flip_frame = ttk.Frame(self.advanced_settings_frame)
        flip_frame.grid(row=5, column=1, padx=5, pady=5, sticky="w")
        self.btn_flip_horizontal = ttk.Button(flip_frame, command=lambda: self.flip_image(horizontal=True))
        self.btn_flip_horizontal.pack(side=tk.LEFT, padx=2)
        self.btn_flip_vertical = ttk.Button(flip_frame, command=lambda: self.flip_image(vertical=True))
        self.btn_flip_vertical.pack(side=tk.LEFT, padx=2)
    
    def create_filter_settings(self):
        # Gaussian Blur
        self.gaussian_frame = ttk.Frame(self.advanced_settings_frame)
        ttk.Label(self.gaussian_frame, text="Kernel:").pack(side=tk.LEFT, padx=5)
        self.gaussian_kernel_var = tk.IntVar(value=5)
        gaussian_slider = ttk.Scale(
            self.gaussian_frame,
            from_=1, to=31,
            variable=self.gaussian_kernel_var,
            command=lambda e: self.process_image()
        )
        gaussian_slider.pack(side=tk.LEFT, fill=tk.X, expand=True)
        
        # Canny Edge
        self.canny_frame = ttk.Frame(self.advanced_settings_frame)
        ttk.Label(self.canny_frame, text="Low:").pack(side=tk.LEFT, padx=5)
        self.canny_thresh1_var = tk.IntVar(value=100)
        canny_thresh1_slider = ttk.Scale(
            self.canny_frame,
            from_=0, to=500,
            variable=self.canny_thresh1_var,
            command=lambda e: self.process_image()
        )
        canny_thresh1_slider.pack(side=tk.LEFT, fill=tk.X, expand=True)
        
        ttk.Label(self.canny_frame, text="High:").pack(side=tk.LEFT, padx=5)
        self.canny_thresh2_var = tk.IntVar(value=200)
        canny_thresh2_slider = ttk.Scale(
            self.canny_frame,
            from_=0, to=500,
            variable=self.canny_thresh2_var,
            command=lambda e: self.process_image()
        )
        canny_thresh2_slider.pack(side=tk.LEFT, fill=tk.X, expand=True)
        
        # قرار دادن این فریم‌ها در گرید (ابتدا پنهان)
        self.gaussian_frame.grid(row=6, column=0, columnspan=2, padx=5, pady=2, sticky="ew")
        self.canny_frame.grid(row=7, column=0, columnspan=2, padx=5, pady=2, sticky="ew")
        
        # پنهان
        self.gaussian_frame.grid_remove()
        self.canny_frame.grid_remove()
    
    def filter_changed(self, event=None):
        selected_filter = self.filter_var.get()
        
        # پنهان کردن همه فریم‌های فیلتر
        self.gaussian_frame.grid_remove()
        self.canny_frame.grid_remove()
        self.threshold_frame.grid_remove()
        
        if selected_filter == "Gaussian Blur":
            self.gaussian_frame.grid()
        elif selected_filter == "Canny Edge":
            self.canny_frame.grid()
        elif selected_filter == "Thresholding":
            self.threshold_frame.grid()
        
        self.process_image()
        
    def load_images(self):
        file_paths = filedialog.askopenfilenames(filetypes=[("Image Files", "*.png *.jpg *.jpeg *.bmp")])
        for path in file_paths:
            img = cv2.imread(path)
            if img is not None:
                title = path.split("/")[-1]
                if title in self.images:
                    messagebox.showwarning(
                        self.i18n[self.current_language]["warning"], 
                        self.i18n[self.current_language]["image_already_loaded"].format(title=title)
                    )
                    continue
                self.add_image_tab(img, title)
            else:
                messagebox.showerror(
                    self.i18n[self.current_language]["error"], 
                    self.i18n[self.current_language]["image_load_failed"].format(path=path)
                )
        
    def add_image_tab(self, image, title):
        frame = ttk.Frame(self.notebook)
        self.notebook.add(frame, text=title)
        
        self.images[title] = {
            "original": image,
            "processed": image.copy(),
            "controls": {}
        }
    
    def tab_changed(self, event):
        current_tab = self.notebook.tab(self.notebook.select(), "text")
        self.current_tab = current_tab
        self.update_display()
        self.update_controls()
        
    def update_display(self):
        if self.current_tab is None:
            return
        
        img_data = self.images[self.current_tab]
        
        original = self.resize_image(img_data["original"], 400)
        self.original_label.configure(image=original)
        self.original_label.image = original
        
        processed = self.resize_image(img_data["processed"], 400)
        self.processed_label.configure(image=processed)
        self.processed_label.image = processed
        
        advanced = self.resize_image(img_data.get("advanced", img_data["processed"]), 400)
        self.advanced_label.configure(image=advanced)
        self.advanced_label.image = advanced
        
        self.show_histogram(img_data["processed"])
    
    def resize_image(self, image, target_size):
        h, w = image.shape[:2]
        ratio = target_size / max(h, w)
        new_size = (int(w*ratio), int(h*ratio))
        
        img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_pil = ImageOps.contain(img_pil, (target_size, target_size))
        return ImageTk.PhotoImage(img_pil)
    
    def show_histogram(self, image):
        fig = Figure(figsize=(4, 2), dpi=100)
        ax = fig.add_subplot(111)
        
        if len(image.shape) == 2:
            ax.hist(image.ravel(), 256, [0,256], color='gray')
            ax.set_xlabel('Intensity')
            ax.set_ylabel('Frequency')
        else:
            colors = ('b','g','r')
            for i, col in enumerate(colors):
                hist = cv2.calcHist([image],[i],None,[256],[0,256])
                ax.plot(hist, color=col, label=f'{col.upper()} channel')
            ax.set_xlabel('Intensity')
            ax.set_ylabel('Frequency')
            ax.legend()
        
        ax.set_xlim([0,256])
        ax.set_title('Histogram')
        ax.grid(True)
        
        if self.hist_canvas:
            self.hist_canvas.get_tk_widget().destroy()
        
        self.hist_canvas = FigureCanvasTkAgg(fig, master=self.hist_frame)
        self.hist_canvas.draw()
        self.hist_canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
    
    def update_controls(self, event=None):
        """ایجاد/به‌روزرسانی اسلایدرهای مخصوص مدل رنگی انتخاب‌شده."""
        for widget in self.settings_frame.winfo_children():
            widget.destroy()
        
        model = self.color_model_var.get()
        img_data = self.images.get(self.current_tab)
        if not img_data or not model:
            return
        
        controls = img_data["controls"].setdefault(model, {})
        
        # بسته به مدل رنگی، اسلایدرهای مختلفی ایجاد می‌کنیم
        if model == "RGB":
            controls["Red"] = tk.IntVar(value=0)
            controls["Green"] = tk.IntVar(value=0)
            controls["Blue"] = tk.IntVar(value=0)
            self.create_slider(self.i18n[self.current_language]["red"], -255, 255, controls["Red"])
            self.create_slider(self.i18n[self.current_language]["green"], -255, 255, controls["Green"])
            self.create_slider(self.i18n[self.current_language]["blue"], -255, 255, controls["Blue"])
        
        elif model == "HSV":
            controls["Hue"] = tk.IntVar(value=0)
            controls["Saturation"] = tk.IntVar(value=0)
            controls["Value"] = tk.IntVar(value=0)
            self.create_slider("Hue", -180, 180, controls["Hue"])
            self.create_slider("Saturation", -255, 255, controls["Saturation"])
            self.create_slider("Value", -255, 255, controls["Value"])
        
        elif model == "LAB":
            controls["L"] = tk.IntVar(value=0)
            controls["A"] = tk.IntVar(value=0)
            controls["B"] = tk.IntVar(value=0)
            self.create_slider("L", -127, 127, controls["L"])
            self.create_slider("A", -127, 127, controls["A"])
            self.create_slider("B", -127, 127, controls["B"])
        
        elif model == "YCbCr":
            controls["Y"] = tk.IntVar(value=0)
            controls["Cb"] = tk.IntVar(value=0)
            controls["Cr"] = tk.IntVar(value=0)
            self.create_slider("Y", -127, 127, controls["Y"])
            self.create_slider("Cb", -127, 127, controls["Cb"])
            self.create_slider("Cr", -127, 127, controls["Cr"])
        
        elif model == "HSL":
            controls["Hue"] = tk.IntVar(value=0)
            controls["Saturation"] = tk.IntVar(value=0)
            controls["Lightness"] = tk.IntVar(value=0)
            self.create_slider("Hue", -180, 180, controls["Hue"])
            self.create_slider("Saturation", -100, 100, controls["Saturation"])
            self.create_slider("Lightness", -100, 100, controls["Lightness"])
        
        elif model == "XYZ":
            controls["X"] = tk.IntVar(value=0)
            controls["Y"] = tk.IntVar(value=0)
            controls["Z"] = tk.IntVar(value=0)
            self.create_slider("X", -255, 255, controls["X"])
            self.create_slider("Y", -255, 255, controls["Y"])
            self.create_slider("Z", -255, 255, controls["Z"])
        
        elif model == "Grayscale":
            controls["Brightness"] = tk.IntVar(value=0)
            controls["Contrast"] = tk.DoubleVar(value=1.0)
            self.create_slider(self.i18n[self.current_language]["brightness"], -255, 255, controls["Brightness"])
            self.create_slider(self.i18n[self.current_language]["contrast"], 0.5, 3.0, controls["Contrast"])
        
        elif model == "CMYK":
            controls["Cyan"] = tk.IntVar(value=0)
            controls["Magenta"] = tk.IntVar(value=0)
            controls["Yellow"] = tk.IntVar(value=0)
            controls["Key"] = tk.IntVar(value=0)
            self.create_slider("Cyan", -100, 100, controls["Cyan"])
            self.create_slider("Magenta", -100, 100, controls["Magenta"])
            self.create_slider("Yellow", -100, 100, controls["Yellow"])
            self.create_slider("Key", -100, 100, controls["Key"])
        
        # دکمهٔ اعمال تغییرات
        self.btn_apply_changes = ttk.Button(
            self.settings_frame, 
            command=self.process_image
        )
        self.btn_apply_changes.pack(pady=5)
        
        # به‌روزرسانی متن دکمه (پس از ساخت)
        self.btn_apply_changes.config(text=self.i18n[self.current_language]["apply_model_changes"])
    
    def create_slider(self, label, from_, to, var):
        frame = ttk.Frame(self.settings_frame)
        frame.pack(fill=tk.X, pady=2)
        
        ttk.Label(frame, text=label, width=12).pack(side=tk.LEFT)
        slider = ttk.Scale(
            frame,
            from_=from_,
            to=to,
            variable=var,
            command=lambda v: self.process_image()
        )
        slider.pack(side=tk.LEFT, fill=tk.X, expand=True)
        ttk.Label(frame, textvariable=var, width=5).pack(side=tk.LEFT)
    
    def process_image(self):
        if self.current_tab is None:
            return
        
        img_data = self.images[self.current_tab]
        model = self.color_model_var.get()
        original = img_data["original"]
        processed = original.copy()
        
        try:
            # اعمال تنظیمات مدل رنگی
            if model == "RGB":
                b, g, r = cv2.split(processed)
                r = np.clip(r + img_data["controls"][model]["Red"].get(), 0, 255).astype(np.uint8)
                g = np.clip(g + img_data["controls"][model]["Green"].get(), 0, 255).astype(np.uint8)
                b = np.clip(b + img_data["controls"][model]["Blue"].get(), 0, 255).astype(np.uint8)
                processed = cv2.merge([b, g, r])
            
            elif model == "HSV":
                hsv = cv2.cvtColor(processed, cv2.COLOR_BGR2HSV).astype(np.float32)
                hsv[..., 0] = np.mod(hsv[..., 0] + img_data["controls"][model]["Hue"].get(), 180)
                hsv[..., 1] = np.clip(hsv[..., 1] + img_data["controls"][model]["Saturation"].get(), 0, 255)
                hsv[..., 2] = np.clip(hsv[..., 2] + img_data["controls"][model]["Value"].get(), 0, 255)
                processed = cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)
            
            elif model == "LAB":
                lab = cv2.cvtColor(processed, cv2.COLOR_BGR2LAB).astype(np.float32)
                lab[..., 0] = np.clip(lab[..., 0] + img_data["controls"][model]["L"].get(), 0, 255)
                lab[..., 1] = np.clip(lab[..., 1] + img_data["controls"][model]["A"].get(), 0, 255)
                lab[..., 2] = np.clip(lab[..., 2] + img_data["controls"][model]["B"].get(), 0, 255)
                processed = cv2.cvtColor(lab.astype(np.uint8), cv2.COLOR_LAB2BGR)
            
            elif model == "YCbCr":
                ycbcr = cv2.cvtColor(processed, cv2.COLOR_BGR2YCrCb).astype(np.float32)
                ycbcr[..., 0] = np.clip(ycbcr[..., 0] + img_data["controls"][model]["Y"].get(), 0, 255)
                ycbcr[..., 1] = np.clip(ycbcr[..., 1] + img_data["controls"][model]["Cb"].get(), 0, 255)
                ycbcr[..., 2] = np.clip(ycbcr[..., 2] + img_data["controls"][model]["Cr"].get(), 0, 255)
                processed = cv2.cvtColor(ycbcr.astype(np.uint8), cv2.COLOR_YCrCb2BGR)
            
            elif model == "HSL":
                hsl = self.bgr2hsl(processed)
                hsl[..., 0] = np.mod(hsl[..., 0] + img_data["controls"][model]["Hue"].get(), 360)
                hsl[..., 1] = np.clip(hsl[..., 1] + img_data["controls"][model]["Saturation"].get(), 0, 100)
                hsl[..., 2] = np.clip(hsl[..., 2] + img_data["controls"][model]["Lightness"].get(), 0, 100)
                processed = self.hsl2bgr(hsl)
            
            elif model == "XYZ":
                xyz = cv2.cvtColor(processed, cv2.COLOR_BGR2XYZ).astype(np.float32)
                xyz[..., 0] = np.clip(xyz[..., 0] + img_data["controls"][model]["X"].get(), 0, 255)
                xyz[..., 1] = np.clip(xyz[..., 1] + img_data["controls"][model]["Y"].get(), 0, 255)
                xyz[..., 2] = np.clip(xyz[..., 2] + img_data["controls"][model]["Z"].get(), 0, 255)
                processed = cv2.cvtColor(xyz.astype(np.uint8), cv2.COLOR_XYZ2BGR)
            
            elif model == "Grayscale":
                gray = cv2.cvtColor(processed, cv2.COLOR_BGR2GRAY)
                contrast = img_data["controls"][model]["Contrast"].get()
                brightness = img_data["controls"][model]["Brightness"].get()
                processed = cv2.convertScaleAbs(gray, alpha=contrast, beta=brightness)
                processed = cv2.cvtColor(processed, cv2.COLOR_GRAY2BGR)
            
            elif model == "CMYK":
                cmyk = self.bgr2cmyk(processed)
                cmyk[..., 0] = np.clip(cmyk[..., 0] + img_data["controls"][model]["Cyan"].get()/100, 0, 1)
                cmyk[..., 1] = np.clip(cmyk[..., 1] + img_data["controls"][model]["Magenta"].get()/100, 0, 1)
                cmyk[..., 2] = np.clip(cmyk[..., 2] + img_data["controls"][model]["Yellow"].get()/100, 0, 1)
                cmyk[..., 3] = np.clip(cmyk[..., 3] + img_data["controls"][model]["Key"].get()/100, 0, 1)
                processed = self.cmyk2bgr(cmyk)
            
            # در نهایت فیلترهای پیشرفته (روشنایی، کنتراست، فیلتر Gaussian و...)
            processed = self.apply_advanced_settings(processed)
            
            img_data["processed"] = processed
            self.update_display()
        
        except Exception as e:
            messagebox.showerror(
                self.i18n[self.current_language]["error"], 
                f"پردازش تصویر ناموفق بود: {str(e)}"
            )
    
    def apply_advanced_settings(self, image):
        # روشنایی
        brightness = self.brightness_var.get()
        image = cv2.convertScaleAbs(image, alpha=1, beta=brightness)
        
        # کنتراست
        contrast = self.contrast_var.get()
        image = cv2.convertScaleAbs(image, alpha=contrast, beta=0)
        
        # فیلتر
        selected_filter = self.filter_var.get()
        if selected_filter == "Gaussian Blur":
            ksize = self.gaussian_kernel_var.get()
            ksize = max(1, int(ksize))
            if ksize % 2 == 0:
                ksize += 1
            image = cv2.GaussianBlur(image, (ksize, ksize), 0)
        elif selected_filter == "Median Blur":
            ksize = self.gaussian_kernel_var.get()
            ksize = max(1, int(ksize))
            if ksize % 2 == 0:
                ksize += 1
            image = cv2.medianBlur(image, ksize)
        elif selected_filter == "Canny Edge":
            thresh1 = self.canny_thresh1_var.get()
            thresh2 = self.canny_thresh2_var.get()
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            edges = cv2.Canny(gray, thresh1, thresh2)
            image = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
        elif selected_filter == "Sharpen":
            kernel = np.array([[0, -1, 0],
                               [-1, 5,-1],
                               [0, -1, 0]])
            image = cv2.filter2D(image, -1, kernel)
        elif selected_filter == "Sepia":
            sepia_filter = np.array([[0.272, 0.534, 0.131],
                                     [0.349, 0.686, 0.168],
                                     [0.393, 0.769, 0.189]])
            image = cv2.transform(image, sepia_filter)
            image = np.clip(image, 0, 255).astype(np.uint8)
        elif selected_filter == "Emboss":
            kernel = np.array([[-2, -1, 0],
                               [-1, 1, 1],
                               [0, 1, 2]])
            image = cv2.filter2D(image, -1, kernel)
        elif selected_filter == "Histogram Equalization":
            if len(image.shape) == 2:
                image = cv2.equalizeHist(image)
                image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
            else:
                ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
                ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])
                image = cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)
        elif selected_filter == "Thresholding":
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            thresh = self.threshold_var.get()
            _, thresh_img = cv2.threshold(gray, thresh, 255, cv2.THRESH_BINARY)
            image = cv2.cvtColor(thresh_img, cv2.COLOR_GRAY2BGR)
        
        return image
    
    def reset_settings(self):
        if self.current_tab:
            img_data = self.images[self.current_tab]
            img_data["processed"] = img_data["original"].copy()
            
            for model in img_data["controls"]:
                for var in img_data["controls"][model].values():
                    if isinstance(var, tk.DoubleVar):
                        var.set(1.0)
                    else:
                        var.set(0)
            
            # ریست تنظیمات پیشرفته
            self.brightness_var.set(0)
            self.contrast_var.set(1.0)
            self.filter_var.set("None")
            self.threshold_var.set(128)
            self.threshold_frame.grid_remove()
            
            # ریست فریم‌های فیلتر
            self.gaussian_kernel_var.set(5)
            self.canny_thresh1_var.set(100)
            self.canny_thresh2_var.set(200)
            self.gaussian_frame.grid_remove()
            self.canny_frame.grid_remove()
            
            self.update_display()
            self.update_controls()
    
    def save_image(self):
        if self.current_tab is None:
            messagebox.showwarning(
                self.i18n[self.current_language]["warning"], 
                self.i18n[self.current_language]["no_image_to_save"]
            )
            return
        
        img_data = self.images[self.current_tab]
        processed = img_data["processed"]
        file_path = filedialog.asksaveasfilename(
            defaultextension=".png",
            filetypes=[("PNG files", "*.png"), ("JPEG files", "*.jpg"), ("BMP files", "*.bmp")],
            initialfile=f"{self.current_tab}_processed"
        )
        if file_path:
            cv2.imwrite(file_path, processed)
            messagebox.showinfo(
                self.i18n[self.current_language]["success"], 
                self.i18n[self.current_language]["image_saved_successfully"].format(file_path=file_path)
            )
    
    # توابع تبدیل رنگ سفارشی
    def bgr2hsl(self, img):
        img = img.astype(np.float32) / 255.0
        hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
        hls[..., 1] *= 100  # Lightness
        hls[..., 2] *= 100  # Saturation
        return hls
    
    def hsl2bgr(self, hsl):
        hsl = hsl.copy()
        hsl[..., 1] /= 100.0
        hsl[..., 2] /= 100.0
        img = cv2.cvtColor(hsl.astype(np.float32), cv2.COLOR_HLS2BGR)
        return (img * 255).astype(np.uint8)
    
    def bgr2cmyk(self, img):
        img = img.astype(np.float32) / 255.0
        K = 1 - np.max(img, axis=2)
        C = (1 - img[..., 2] - K) / (1 - K + 1e-7)
        M = (1 - img[..., 1] - K) / (1 - K + 1e-7)
        Y = (1 - img[..., 0] - K) / (1 - K + 1e-7)
        return np.stack([C, M, Y, K], axis=2)
    
    def cmyk2bgr(self, cmyk):
        C, M, Y, K = cmyk[..., 0], cmyk[..., 1], cmyk[..., 2], cmyk[..., 3]
        R = (1 - C) * (1 - K)
        G = (1 - M) * (1 - K)
        B = (1 - Y) * (1 - K)
        img = np.stack([B, G, R], axis=2)
        return (img * 255).astype(np.uint8)
    
    def rotate_image(self, angle):
        if self.current_tab is None:
            return
        img_data = self.images[self.current_tab]
        rotate_codes = {90: cv2.ROTATE_90_CLOCKWISE,
                        180: cv2.ROTATE_180,
                        270: cv2.ROTATE_90_COUNTERCLOCKWISE}
        if angle in rotate_codes:
            rotated = cv2.rotate(img_data["processed"], rotate_codes[angle])
            img_data["processed"] = rotated
            self.update_display()
    
    def flip_image(self, horizontal=False, vertical=False):
        if self.current_tab is None:
            return
        img_data = self.images[self.current_tab]
        if horizontal and vertical:
            flip_code = -1
        elif horizontal:
            flip_code = 1
        elif vertical:
            flip_code = 0
        else:
            return
        flipped = cv2.flip(img_data["processed"], flip_code)
        img_data["processed"] = flipped
        self.update_display()

if __name__ == "__main__":
    root = tk.Tk()
    app = AdvancedImageProcessor(root)
    root.mainloop()
